In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [52]:
from datasets import load_dataset
import pandas as pd
import numpy as np
from collections import Counter
from itertools import combinations
from sklearn.feature_extraction.text import TfidfVectorizer
import json

In [33]:
dataset = load_dataset("AkashPS11/recipes_data_food.com")
df = pd.DataFrame(dataset['train'])
df.head()

,RecipeId,Barcode,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38.0,*38*,Low-Fat Berry Blue Frozen Dessert,1533.0,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,...,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,None,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,39.0,*39*,Biryani,1567.0,elly9812,PT25M,PT4H,PT4H25M,1999-08-29T13:12:00Z,Make and share this Biryani recipe from Food.com.,...,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,None,"c(""Soak saffron in warm milk for 5 minutes and..."
2,40.0,*40*,Best Lemonade,1566.0,Stephen Little,PT5M,PT30M,PT35M,1999-09-05T19:52:00Z,This is from one of my first Good House Keepi...,...,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,None,"c(""Into a 1 quart Jar with tight fitting lid, ..."
3,41.0,*41*,Carina's Tofu-Vegetable Kebabs,1586.0,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03T14:54:00Z,This dish is best prepared a day in advance to...,...,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"c(""Drain the tofu, carefully squeezing out exc..."
4,42.0,*42*,Cabbage Soup,1538.0,Duckie067,PT30M,PT20M,PT50M,1999-09-19T06:19:00Z,Make and share this Cabbage Soup recipe from F...,...,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,None,"c(""Mix everything together and bring to a boil..."


In [34]:
print(df.columns)

Index(['RecipeId', 'Barcode', 'Name', 'AuthorId', 'AuthorName', 'CookTime',
       'PrepTime', 'TotalTime', 'DatePublished', 'Description', 'Images',
       'RecipeCategory', 'Keywords', 'RecipeIngredientQuantities',
       'RecipeIngredientParts', 'AggregatedRating', 'ReviewCount', 'Calories',
       'FatContent', 'SaturatedFatContent', 'CholesterolContent',
       'SodiumContent', 'CarbohydrateContent', 'FiberContent', 'SugarContent',
       'ProteinContent', 'RecipeServings', 'RecipeYield',
       'RecipeInstructions'],
      dtype='object')


In [35]:
print(df.dtypes)

RecipeId                      float64
Barcode                        object
Name                           object
AuthorId                      float64
AuthorName                     object
CookTime                       object
PrepTime                       object
TotalTime                      object
DatePublished                  object
Description                    object
Images                         object
RecipeCategory                 object
Keywords                       object
RecipeIngredientQuantities     object
RecipeIngredientParts          object
AggregatedRating              float64
ReviewCount                   float64
Calories                      float64
FatContent                    float64
SaturatedFatContent           float64
CholesterolContent            float64
SodiumContent                 float64
CarbohydrateContent           float64
FiberContent                  float64
SugarContent                  float64
ProteinContent                float64
RecipeServin

In [36]:
print(df.isnull().sum())

RecipeId                      1047315
Barcode                        521340
Name                          1047315
AuthorId                      1047315
AuthorName                    1047315
CookTime                      1047818
PrepTime                      1047315
TotalTime                     1047315
DatePublished                 1047315
Description                   1047315
Images                        1047315
RecipeCategory                1047315
Keywords                      1047328
RecipeIngredientQuantities    1047316
RecipeIngredientParts         1047315
AggregatedRating              1047560
ReviewCount                   1047547
Calories                      1047315
FatContent                    1047315
SaturatedFatContent           1047315
CholesterolContent            1047315
SodiumContent                 1047315
CarbohydrateContent           1047315
FiberContent                  1047315
SugarContent                  1047315
ProteinContent                1047315
RecipeServin

In [37]:
print(f"Number of rows: {len(df)}")

Number of rows: 1048543


In [38]:
df = df.drop(columns=["Barcode", "AuthorId", "AuthorName", "DatePublished", "Images",])

In [39]:
df = df.dropna(how="all")
print(df.isnull().sum())

RecipeId                        0
Name                            0
CookTime                      503
PrepTime                        0
TotalTime                       0
Description                     0
RecipeCategory                  0
Keywords                       13
RecipeIngredientQuantities      1
RecipeIngredientParts           0
AggregatedRating              245
ReviewCount                   232
Calories                        0
FatContent                      0
SaturatedFatContent             0
CholesterolContent              0
SodiumContent                   0
CarbohydrateContent             0
FiberContent                    0
SugarContent                    0
ProteinContent                  0
RecipeServings                405
RecipeYield                   839
RecipeInstructions              0
dtype: int64


In [40]:
print(df.RecipeIngredientParts.head())

0    c("blueberries", "granulated sugar", "vanilla ...
1    c("saffron", "milk", "hot green chili peppers"...
2    c("sugar", "lemons, rind of", "lemon, zest of"...
3    c("extra firm tofu", "eggplant", "zucchini", "...
4    c("plain tomato juice", "cabbage", "onion", "c...
Name: RecipeIngredientParts, dtype: object


In [41]:
def normalize_ingredients(ingredient_list):
  """Removes leading 'c(' and trailing ')' and converts to a list of strings."""
  if isinstance(ingredient_list, str) and ingredient_list.startswith("c(") and ingredient_list.endswith(")"):
    cleaned_string = ingredient_list[2:-1]
    #split by ", " to handle multiple ingredients
    ingredients = [ingredient.strip().strip('"') for ingredient in cleaned_string.split('", "')]
    return ingredients
  return ingredient_list

df.loc[:, "RecipeIngredientParts"] = df["RecipeIngredientParts"].apply(normalize_ingredients)

print(df.RecipeIngredientParts.head())


0    [blueberries, granulated sugar, vanilla yogurt...
1    [saffron, milk, hot green chili peppers, onion...
2    [sugar, lemons, rind of, lemon, zest of, fresh...
3    [extra firm tofu, eggplant, zucchini, mushroom...
4    [plain tomato juice, cabbage, onion, carrots, ...
Name: RecipeIngredientParts, dtype: object


In [42]:
#Applying the normalization to keywords, recipeingredientquantities,
#Both have the weird formatting whose existence i dont understand
# Apply transformations safely using .loc to avoid SettingWithCopyWarning
df.loc[:, "Keywords"] = df["Keywords"].apply(normalize_ingredients)
print(df.Keywords.head())
df.loc[:, "RecipeIngredientQuantities"] = df["RecipeIngredientQuantities"].apply(normalize_ingredients)
print(df.RecipeIngredientQuantities.head())
df.loc[:, "RecipeInstructions"] = df["RecipeInstructions"].apply(normalize_ingredients)
print(df.RecipeInstructions.head())


0    [Dessert, Low Protein, Low Cholesterol, Health...
1    [Chicken Thigh & Leg, Chicken, Poultry, Meat, ...
2    [Low Protein, Low Cholesterol, Healthy, Summer...
3    [Beans, Vegetable, Low Cholesterol, Weeknight,...
4    [Low Protein, Vegan, Low Cholesterol, Healthy,...
Name: Keywords, dtype: object
0                                       [4, 1/4, 1, 1]
1    [1, 4, 2, 2, 8, 1/4, 8, 1/2, 1, 1, 1/4, 1/4, 1...
2                   [1 1/2, 1", NA, "1 1/2", NA, "3/4]
3    [12, 1, 2, 1, 10, 1, 3, 2, 2, 2, 1, 2, 1/2, 1/...
4                                     [46, 4, 1, 2, 1]
Name: RecipeIngredientQuantities, dtype: object
0    [Toss 2 cups berries with sugar., Let stand fo...
1    [Soak saffron in warm milk for 5 minutes and p...
2    [Into a 1 quart Jar with tight fitting lid, pu...
3    [Drain the tofu, carefully squeezing out exces...
4    [Mix everything together and bring to a boil.,...
Name: RecipeInstructions, dtype: object


#Stuff below generated by chat gpt

In [43]:
#Standardizing synonyms
synonyms = {
    # 🥖 Baking Ingredients
    "baking soda": "sodium bicarbonate",
    "baking powder": "baking powder",
    "cornstarch": "corn flour",
    "powdered sugar": "confectioners sugar",
    "granulated sugar": "white sugar",
    "brown sugar": "raw sugar",
    "caster sugar": "superfine sugar",
    "molasses": "treacle",
    "honey": "natural sweetener",
    "maple syrup": "tree syrup",
    "agave nectar": "agave syrup",
    "vanilla extract": "vanilla essence",
    "cocoa powder": "unsweetened cocoa",
    "dark chocolate": "bittersweet chocolate",
    "milk chocolate": "sweet chocolate",
    "white chocolate": "cocoa butter chocolate",

    # 🥛 Dairy & Dairy Substitutes
    "whole milk": "full-fat milk",
    "skim milk": "fat-free milk",
    "buttermilk": "cultured milk",
    "heavy cream": "whipping cream",
    "sour cream": "fermented cream",
    "cream cheese": "soft cheese spread",
    "cheddar": "sharp cheese",
    "parmesan": "parmigiano-reggiano",
    "mozzarella": "pizza cheese",
    "gouda": "smoked cheese",
    "swiss cheese": "emmental cheese",
    "feta": "greek cheese",
    "goat cheese": "chèvre",
    "yogurt": "curd",
    "greek yogurt": "strained yogurt",
    "almond milk": "nut milk",
    "oat milk": "plant-based milk",
    "soy milk": "legume milk",
    "coconut milk": "tropical milk",

    # 🍗 Meat & Protein
    "ground beef": "minced beef",
    "ground pork": "minced pork",
    "ground turkey": "minced turkey",
    "ground chicken": "minced chicken",
    "bacon": "pork belly strips",
    "ham": "cured pork",
    "salmon": "smoked salmon",
    "shrimp": "prawns",
    "crab": "crustacean meat",
    "lamb": "mutton",
    "duck": "poultry",
    "turkey": "large poultry",
    "venison": "deer meat",
    "tofu": "bean curd",
    "tempeh": "fermented soybeans",
    "seitan": "wheat gluten",
    "chickpeas": "garbanzo beans",

    # 🥬 Vegetables
    "scallions": "green onions",
    "zucchini": "courgette",
    "eggplant": "aubergine",
    "cilantro": "coriander leaves",
    "bell pepper": "capsicum",
    "beetroot": "beet",
    "arugula": "rocket",
    "chickpeas": "garbanzo beans",
    "fennel": "sweet anise",
    "snow peas": "mangetout",
    "potato": "starchy tuber",
    "sweet potato": "yam",
    "butternut squash": "winter squash",
    "spaghetti squash": "string squash",
    "carrot": "root vegetable",
    "radish": "spicy root",
    "cabbage": "cole vegetable",
    "kale": "leafy green",
    "spinach": "leafy vegetable",
    "romaine": "cos lettuce",
    "lettuce": "leafy green",
    "cucumber": "gourd vegetable",
    "tomato": "nightshade fruit",
    "avocado": "butter fruit",

    # 🍎 Fruits
    "raisins": "dried grapes",
    "dates": "pitted dates",
    "coconut": "shredded coconut",
    "lime": "green citrus",
    "lemon": "yellow citrus",
    "blueberries": "wild berries",
    "strawberries": "red berries",
    "oranges": "citrus fruit",
    "pineapple": "tropical fruit",
    "grapes": "vine fruit",
    "mango": "stone fruit",
    "papaya": "pawpaw",
    "banana": "yellow fruit",
    "watermelon": "summer melon",
    "peach": "stone fruit",
    "plum": "drupe",
    "cherry": "tiny stone fruit",

    # 🌾 Grains & Legumes
    "white rice": "long grain rice",
    "brown rice": "whole grain rice",
    "quinoa": "super grain",
    "oats": "rolled oats",
    "barley": "hulled barley",
    "couscous": "durum wheat granules",
    "lentils": "dal",
    "black beans": "turtle beans",
    "pinto beans": "frijoles",
    "chickpeas": "garbanzo beans",
    "kidney beans": "red beans",
    "soybeans": "edamame",

    # 🌿 Herbs & Spices
    "chili powder": "red pepper powder",
    "cinnamon": "ground cinnamon bark",
    "paprika": "red bell pepper powder",
    "cumin": "jeera",
    "coriander": "dhania",
    "turmeric": "haldi",
    "cloves": "dried flower buds",
    "nutmeg": "mace spice",
    "star anise": "anise spice",
    "garlic powder": "dehydrated garlic",
    "onion powder": "dehydrated onion",
    "black pepper": "ground peppercorns",
    "white pepper": "mild ground peppercorns",
    "oregano": "wild marjoram",
    "thyme": "woody herb",
    "basil": "sweet basil",
    "rosemary": "needle herb",

    # 🥫 Condiments & Sauces
    "soy sauce": "shoyu",
    "tamari": "gluten-free soy sauce",
    "fish sauce": "nam pla",
    "worcestershire sauce": "fermented sauce",
    "hot sauce": "spicy condiment",
    "ketchup": "tomato sauce",
    "mayonnaise": "egg-based dressing",
    "mustard": "yellow mustard",
    "vinegar": "acidic condiment",
    "balsamic vinegar": "aged vinegar",

    # 🥜 Nuts & Seeds
    "peanuts": "groundnuts",
    "almonds": "blanched almonds",
    "cashews": "creamy nuts",
    "walnuts": "brain nuts",
    "chia seeds": "super seeds",
    "flaxseeds": "linseeds",
    "pumpkin seeds": "pepitas",
    "sesame seeds": "tahini base",

    # 🍶 Oils & Fats
    "vegetable oil": "neutral oil",
    "canola oil": "rapeseed oil",
    "olive oil": "extra virgin olive oil",
    "butter": "dairy fat",
    "margarine": "butter substitute",
    "shortening": "hydrogenated fat",
    "ghee": "clarified butter",
    "lard": "pork fat",

    # 🍷 Alcohol & Extracts
    "rum": "sugarcane liquor",
    "whiskey": "aged grain liquor",
    "red wine": "grape-based alcohol",
    "white wine": "dry wine",
    "cognac": "aged brandy",

    # 🍲 Miscellaneous
    "stock": "broth",
    "gelatin": "collagen powder",
    "agar agar": "vegetarian gelatin",
    "arrowroot powder": "thickening starch",
    "yeast": "fermenting agent",
    "panko": "japanese breadcrumbs",
    "breadcrumbs": "crumbed bread",
    "tortilla": "flatbread",
    "pita bread": "pocket bread",
}


In [44]:
#Standardizing units
unit_synonyms = {
    # 🏋️ Weight Measurements
    "g": "grams",
    "gram": "grams",
    "grams": "grams",
    "kg": "kilograms",
    "kilogram": "kilograms",
    "kilograms": "kilograms",
    "mg": "milligrams",
    "milligram": "milligrams",
    "milligrams": "milligrams",
    "oz": "ounces",
    "ounce": "ounces",
    "ounces": "ounces",
    "lb": "pounds",
    "lbs": "pounds",
    "pound": "pounds",
    "pounds": "pounds",

    # 💧 Volume Measurements
    "ml": "milliliters",
    "milliliter": "milliliters",
    "milliliters": "milliliters",
    "l": "liters",
    "liter": "liters",
    "liters": "liters",
    "fl oz": "fluid ounces",
    "fluid ounce": "fluid ounces",
    "fluid ounces": "fluid ounces",
    "cup": "cups",
    "cups": "cups",
    "pt": "pints",
    "pint": "pints",
    "pints": "pints",
    "qt": "quarts",
    "quart": "quarts",
    "quarts": "quarts",
    "gal": "gallons",
    "gallon": "gallons",
    "gallons": "gallons",

    # 🥄 Spoon Measurements
    "tsp": "teaspoons",
    "teaspoon": "teaspoons",
    "teaspoons": "teaspoons",
    "tbsp": "tablespoons",
    "tablespoon": "tablespoons",
    "tablespoons": "tablespoons",
    "dash": "small amount",
    "pinch": "small amount",
    "smidgen": "tiny amount",

    # 🥄 Other Small Volume Units
    "dr": "drams",
    "dram": "drams",
    "drams": "drams",
    "drop": "drops",
    "drops": "drops",

    # ⚖️ Non-Specific Quantities
    "each": "each",
    "piece": "each",
    "pieces": "each",
    "unit": "each",
    "whole": "each",
    "clove": "cloves",
    "cloves": "cloves",
    "stick": "sticks",
    "sticks": "sticks",
    "head": "heads",
    "heads": "heads",
    "bunch": "bunches",
    "bunches": "bunches",

    # 🌡 Temperature Units
    "c": "celsius",
    "°c": "celsius",
    "celsius": "celsius",
    "f": "fahrenheit",
    "°f": "fahrenheit",
    "fahrenheit": "fahrenheit",

    # ⏲️ Time Units
    "s": "seconds",
    "sec": "seconds",
    "second": "seconds",
    "seconds": "seconds",
    "m": "minutes",
    "min": "minutes",
    "minute": "minutes",
    "minutes": "minutes",
    "h": "hours",
    "hr": "hours",
    "hour": "hours",
    "hours": "hours",

    # 🍞 Baking Specific Units
    "loaf": "loaves",
    "loaves": "loaves",
    "slice": "slices",
    "slices": "slices",
    "roll": "rolls",
    "rolls": "rolls",
    "sheet": "sheets",
    "sheets": "sheets",

    # 🍗 Meat-Specific Measurements
    "fillet": "fillets",
    "fillets": "fillets",
    "cutlet": "cutlets",
    "cutlets": "cutlets",
    "steak": "steaks",
    "steaks": "steaks",
    "rack": "racks",
    "racks": "racks",
    "roast": "roasts",
    "roasts": "roasts",

    # 🏺 Miscellaneous Units
    "bar": "bars",
    "bars": "bars",
    "square": "squares",
    "squares": "squares",
    "can": "cans",
    "cans": "cans",
    "bottle": "bottles",
    "bottles": "bottles",
    "packet": "packets",
    "packets": "packets",
    "bag": "bags",
    "bags": "bags",
    "carton": "cartons",
    "cartons": "cartons",
    "box": "boxes",
    "boxes": "boxes",
    "tube": "tubes",
    "tubes": "tubes",
    "sprig": "sprigs",
    "sprigs": "sprigs"
}


In [45]:
#Standardizing ingredient patterns
import re

ingredient_patterns = [
    # 🔪 Common Preparation Methods (Remove descriptors)
    (r"\bdiced\b", ""),      # Remove "diced"
    (r"\bchopped\b", ""),    # Remove "chopped"
    (r"\bminced\b", ""),     # Remove "minced"
    (r"\bsliced\b", ""),     # Remove "sliced"
    (r"\bgrated\b", ""),     # Remove "grated"
    (r"\bshredded\b", ""),   # Remove "shredded"
    (r"\bcrushed\b", ""),    # Remove "crushed"
    (r"\bmashed\b", ""),     # Remove "mashed"
    (r"\bpeeled\b", ""),     # Remove "peeled"
    (r"\bchopped fine\b", ""), # Remove "chopped fine"
    (r"\bfreshly grated\b", ""), # Remove "freshly grated"
    (r"\bfreshly squeezed\b", ""), # Remove "freshly squeezed"

    # 🏭 Processing & States (Standardize ingredient name)
    (r"\bcanned\b", ""),     # Remove "canned"
    (r"\bjarred\b", ""),     # Remove "jarred"
    (r"\bfrozen\b", ""),     # Remove "frozen"
    (r"\bdried\b", ""),      # Remove "dried"
    (r"\bfresh\b", ""),      # Remove "fresh"
    (r"\braw\b", ""),        # Remove "raw"
    (r"\bcooked\b", ""),     # Remove "cooked"
    (r"\bpre-cooked\b", ""), # Remove "pre-cooked"
    (r"\broasted\b", ""),    # Remove "roasted"
    (r"\btoasted\b", ""),    # Remove "toasted"
    (r"\bboiled\b", ""),     # Remove "boiled"
    (r"\bfried\b", ""),      # Remove "fried"
    (r"\bsmoked\b", ""),     # Remove "smoked"
    (r"\bpickled\b", ""),    # Remove "pickled"
    (r"\bfermented\b", ""),  # Remove "fermented"

    # 🔥 Cooking-Specific Descriptors
    (r"\bgrilled\b", ""),    # Remove "grilled"
    (r"\bbaked\b", ""),      # Remove "baked"
    (r"\bsautéed\b", ""),    # Remove "sautéed"
    (r"\bsteamed\b", ""),    # Remove "steamed"
    (r"\bseared\b", ""),     # Remove "seared"

    # 📦 Packaging & Sizing
    (r"\bpackage\b", ""),    # Remove "package"
    (r"\bpack\b", ""),       # Remove "pack"
    (r"\bbottle\b", ""),     # Remove "bottle"
    (r"\bcan\b", ""),        # Remove "can"
    (r"\bjar\b", ""),        # Remove "jar"
    (r"\bbox\b", ""),        # Remove "box"
    (r"\bbag\b", ""),        # Remove "bag"
    (r"\btube\b", ""),       # Remove "tube"
    (r"\bcarton\b", ""),     # Remove "carton"
    (r"\bcase\b", ""),       # Remove "case"

    # 🏷 Brand-Related & Marketing Terms
    (r"\borganic\b", ""),    # Remove "organic"
    (r"\bnon-GMO\b", ""),    # Remove "non-GMO"
    (r"\bgluten-free\b", ""),# Remove "gluten-free"
    (r"\blow-fat\b", ""),    # Remove "low-fat"
    (r"\blow sodium\b", ""), # Remove "low sodium"
    (r"\bfat-free\b", ""),   # Remove "fat-free"
    (r"\bsugar-free\b", ""), # Remove "sugar-free"
    (r"\breduced sodium\b", ""), # Remove "reduced sodium"

    # ⚖️ Weight & Volume Descriptors
    (r"\bsmall\b", ""),      # Remove "small"
    (r"\bmedium\b", ""),     # Remove "medium"
    (r"\blarge\b", ""),      # Remove "large"
    (r"\bextra-large\b", ""),# Remove "extra-large"

    # 🍞 Texture & Consistency
    (r"\bwhole\b", ""),      # Remove "whole"
    (r"\bcrushed\b", ""),    # Remove "crushed"
    (r"\bfinely ground\b", ""), # Remove "finely ground"
    (r"\bcoarse ground\b", ""), # Remove "coarse ground"
    (r"\bsoftened\b", ""),   # Remove "softened"
    (r"\bmelted\b", ""),     # Remove "melted"
    (r"\bthick-cut\b", ""),  # Remove "thick-cut"
    (r"\bthin-cut\b", ""),   # Remove "thin-cut"

    # 🍶 Liquids & Extracts
    (r"\breduced\b", ""),    # Remove "reduced"
    (r"\bconcentrated\b", ""), # Remove "concentrated"
    (r"\bdiluted\b", ""),    # Remove "diluted"

    # 🏺 Miscellaneous
    (r"\bwith skin\b", ""),  # Remove "with skin"
    (r"\bwithout skin\b", ""), # Remove "without skin"
    (r"\bboneless\b", ""),   # Remove "boneless"
    (r"\bbone-in\b", ""),    # Remove "bone-in"
    (r"\bskin-on\b", ""),    # Remove "skin-on"
    (r"\bskinless\b", ""),   # Remove "skinless"
]


In [46]:
#gitting rid of cleanup patterns
ingredient_cleanup_patterns = [
    # 🍋 Citrus & Fruit Parts
    (r"\bzest of\b", "zest"),  # "zest of lemon" → "lemon zest"
    (r"\brind of\b", "rind"),  # "rind of lemon" → "lemon rind"
    (r"\bpeel of\b", "peel"),  # "peel of orange" → "orange peel"
    (r"\bjuice of\b", "juice"),  # "juice of lime" → "lime juice"
    (r"\bpulp of\b", "pulp"),  # "pulp of passionfruit" → "passionfruit pulp"

    # 🥚 Egg & Dairy Specific
    (r"\bwhites of\b", "egg whites"),  # "whites of eggs" → "egg whites"
    (r"\byolks of\b", "egg yolks"),  # "yolks of eggs" → "egg yolks"
    (r"\bcurds of\b", "curds"),  # "curds of milk" → "milk curds"
    (r"\bwhey of\b", "whey"),  # "whey of milk" → "milk whey"

    # 🥩 Meat & Bone Components
    (r"\bfat of\b", "fat"),  # "fat of pork" → "pork fat"
    (r"\bmarrow of\b", "marrow"),  # "marrow of beef bones" → "beef marrow"
    (r"\bskin of\b", "skin"),  # "skin of chicken" → "chicken skin"
    (r"\bmeat of\b", "meat"),  # "meat of crab" → "crab meat"

    # 🥦 Vegetables & Legumes
    (r"\bseeds of\b", "seeds"),  # "seeds of pumpkin" → "pumpkin seeds"
    (r"\bleaves of\b", "leaves"),  # "leaves of basil" → "basil leaves"
    (r"\bstalk of\b", "stalk"),  # "stalk of celery" → "celery stalk"
    (r"\bstems of\b", "stems"),  # "stems of parsley" → "parsley stems"
    (r"\broots of\b", "roots"),  # "roots of ginger" → "ginger roots"

    # 🍞 Bread & Grain Components
    (r"\bcrust of\b", "crust"),  # "crust of bread" → "bread crust"
    (r"\bcorn of\b", "corn kernels"),  # "corn of cob" → "corn kernels"

    # 🍫 Nuts & Cocoa Derivatives
    (r"\bhusk of\b", "husk"),  # "husk of coconut" → "coconut husk"
    (r"\bshell of\b", "shell"),  # "shell of almonds" → "almond shell"
    (r"\bgrounds of\b", "coffee grounds"),  # "grounds of coffee" → "coffee grounds"
    (r"\bbutter of\b", "butter"),  # "butter of peanuts" → "peanut butter"

    # 🌿 Herbs & Spices
    (r"\bstems of\b", "stems"),  # "stems of cilantro" → "cilantro stems"
    (r"\bpods of\b", "pods"),  # "pods of vanilla" → "vanilla pods"
    (r"\bhusk of\b", "husk"),  # "husk of cardamom" → "cardamom husk"

    # 🌊 Seafood Components
    (r"\broe of\b", "roe"),  # "roe of salmon" → "salmon roe"
    (r"\bshell of\b", "shell"),  # "shell of shrimp" → "shrimp shell"

    # 🏺 Miscellaneous
    (r"\bessence of\b", "extract"),  # "essence of vanilla" → "vanilla extract"
    (r"\boil of\b", "oil"),  # "oil of olives" → "olive oil"
]


#My code again

In [47]:

def preprocess_ingredient(ingredient):
    """Fully processes ingredients using synonym replacement, unit standardization, and cleanup"""

    # 1. Lowercase and strip spaces
    ingredient = ingredient.lower().strip()

    # 2. Synonym Replacement (Whole Word Matches Only)
    words = ingredient.split()  # Split into words
    words = [synonyms[word] if word in synonyms else word for word in words]
    ingredient = " ".join(words)  # Reconstruct ingredient

    # 3. Unit Standardization (Whole Words Only)
    words = ingredient.split()  # Split again for unit replacements
    words = [unit_synonyms[word] if word in unit_synonyms else word for word in words]
    ingredient = " ".join(words)

    # 4. Ingredient Normalization (Regex Cleaning, Applied in Order)
    for pattern, replacement in ingredient_patterns:
        ingredient = re.sub(rf"\b{pattern}\b", replacement, ingredient).strip()

    # 5. Ingredient Cleanup (Final Fixes)
    for pattern, replacement in ingredient_cleanup_patterns:
        ingredient = re.sub(rf"\b{pattern}\b", replacement, ingredient).strip()

    return ingredient


In [48]:
#Apply that goodness
df['RecipeIngredientParts'] = df['RecipeIngredientParts'].apply(lambda ingredients: [preprocess_ingredient(ing) for ing in ingredients])

df['RecipeCategory'] = df['RecipeCategory'].apply(preprocess_ingredient)

# Apply preprocessing to each element inside the list for 'Keywords'
df['Keywords'] = df['Keywords'].apply(lambda lst: [preprocess_ingredient(word) for word in lst] if isinstance(lst, list) else lst)

In [49]:
print(df.head())

   RecipeId                               Name CookTime PrepTime TotalTime  \
0      38.0  Low-Fat Berry Blue Frozen Dessert    PT24H    PT45M  PT24H45M   
1      39.0                            Biryani    PT25M     PT4H   PT4H25M   
2      40.0                      Best Lemonade     PT5M    PT30M     PT35M   
3      41.0     Carina's Tofu-Vegetable Kebabs    PT20M    PT24H  PT24H20M   
4      42.0                       Cabbage Soup    PT30M    PT20M     PT50M   

                                         Description  RecipeCategory  \
0  Make and share this Low-Fat Berry Blue Frozen ...        desserts   
1  Make and share this Biryani recipe from Food.com.  chicken breast   
2  This is from one of my  first Good House Keepi...       beverages   
3  This dish is best prepared a day in advance to...        soy/tofu   
4  Make and share this Cabbage Soup recipe from F...       vegetable   

                                            Keywords  \
0  [dessert, low protein, low cholesterol,

In [50]:
import re
#Makes recipes easier for llm to understand
def format_recipe_instructions(instructions):
    """Formats raw recipe instructions into clean, numbered steps."""
    if isinstance(instructions, list):
        # If instructions are already split into a list, number them
        return [f"Step {i+1}: {step.strip().capitalize()}" for i, step in enumerate(instructions)]
    elif isinstance(instructions, str):
        # Split on common step indicators (periods, new lines, numbers)
        steps = re.split(r'(?<=\.)\s+|\n+|\d+\.\s+', instructions)
        steps = [step.strip() for step in steps if step.strip()]
        return [f"Step {i+1}: {step.capitalize()}" for i, step in enumerate(steps)]
    return instructions  # Return as is if it's not in a recognized format

# Apply formatting to the dataset
df["RecipeInstructions"] = df["RecipeInstructions"].apply(format_recipe_instructions)

# Preview formatted instructions
df["RecipeInstructions"].head()


,RecipeInstructions
0,"[Step 1: Toss 2 cups berries with sugar., Step..."
1,[Step 1: Soak saffron in warm milk for 5 minut...
2,[Step 1: Into a 1 quart jar with tight fitting...
3,"[Step 1: Drain the tofu, carefully squeezing o..."
4,[Step 1: Mix everything together and bring to ...


In [51]:
print(df.columns)

Index(['RecipeId', 'Name', 'CookTime', 'PrepTime', 'TotalTime', 'Description',
       'RecipeCategory', 'Keywords', 'RecipeIngredientQuantities',
       'RecipeIngredientParts', 'AggregatedRating', 'ReviewCount', 'Calories',
       'FatContent', 'SaturatedFatContent', 'CholesterolContent',
       'SodiumContent', 'CarbohydrateContent', 'FiberContent', 'SugarContent',
       'ProteinContent', 'RecipeServings', 'RecipeYield',
       'RecipeInstructions'],
      dtype='object')


In [55]:
# Function to create Ingredients → Recipe Names dataset
def create_ingredient_to_recipe_dataset(df):
    """Generates a dataset mapping available ingredients to possible recipe names."""
    data = []

    for _, row in df.iterrows():
        ingredients = ", ".join(row["RecipeIngredientParts"])  # List of ingredients
        recipe_name = row["Name"]  # Recipe title

        # Format for fine-tuning
        prompt = f"What can I cook with {ingredients}?"
        completion = recipe_name

        data.append({"prompt": prompt, "completion": completion})

    return data


In [57]:
# Function to create Recipe Names → Recipe Instructions dataset
def create_recipe_to_instructions_dataset(df):
    """Generates a dataset mapping recipe names to full instructions."""
    data = []

    for _, row in df.iterrows():
        recipe_name = row["Name"]  # Recipe title
        instructions = " ".join(row["RecipeInstructions"])  # Convert step-by-step instructions into a single string

        # Format for fine-tuning
        prompt = f"How do I make {recipe_name}?"
        completion = instructions

        data.append({"prompt": prompt, "completion": completion})

    return data

In [58]:
# Generate both datasets
ingredient_to_recipe_data = create_ingredient_to_recipe_dataset(df)
recipe_to_instructions_data = create_recipe_to_instructions_dataset(df)


In [61]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [69]:
# Save datasets to Google Drive
json_path = "/content/drive/MyDrive/recipe_generator_model/"

with open(json_path + "ingredients_to_recipe_names.json", "w") as f:
    json.dump(ingredient_to_recipe_data, f, indent=4)

with open(json_path + "recipe_names_to_instructions.json", "w") as f:
    json.dump(recipe_to_instructions_data, f, indent=4)

print(f"✅ Dataset conversion complete! JSON files saved in {json_path}")

✅ Dataset conversion complete! JSON files saved in /content/drive/MyDrive/recipe_generator_model/
